In [1]:
import json
import imgkit
import pandas as pd
import re
import cohere
import random
import base64
from io import BytesIO
import io
from PIL import Image

In [45]:
client = cohere.ClientV2("R8fQH9pZzw70Ixq3eOmcLKiCaZVS0wHs7eUc82dU", base_url="https://stg.api.cohere.ai")

PROMPT =  """Original Text: 
{raw_text}\n

Translation: 
{translation}\n

Instruction:
Given the original text and its translation, improve the quality of the translation by rephrasing it. 
Ensure the rephrased translation closely aligns with the original text in meaning, structure, tone, and style. 
Make the rephrased translation sound natural and fluent in the target language while preserving the core message, correcting any grammatical errors, and retaining all stylistic elements (e.g., enumeration, punctuation, capitalization, spacing, line breaks, etc.) from the original.

The output must strictly follow this format:
Rephrased Translation: <rephrased translation placeholder>"""

In [2]:
with open('table_data/finqa_train.json') as f:
    finqa = json.load(f)

In [3]:
data = finqa[10]['table']

In [4]:
data

[['in millions',
  'defined benefit pension plans',
  'other postretirement benefit plans gross payments',
  'medicare subsidy receipts',
  'postemployment benefit plans'],
 ['2009', '$ 176.3', '$ 56.0', '$ -6.1 ( 6.1 )', '$ 16.6'],
 ['2010', '182.5', '59.9', '-6.7 ( 6.7 )', '17.5'],
 ['2011', '189.8', '63.3', '-7.3 ( 7.3 )', '18.1'],
 ['2012', '197.5', '67.0', '-8.0 ( 8.0 )', '18.8'],
 ['2013', '206.6', '71.7', '-8.7 ( 8.7 )', '19.4'],
 ['2014 2013 2018', '1187.3', '406.8', '-55.3 ( 55.3 )', '106.3']]

In [35]:
from google.cloud import translate_v3
import time

PROJECT_ID = "valued-sight-253418"

def translate_text(text, target_language_code):
    """Translating Text."""

    client = translate_v3.TranslationServiceClient()

    parent = f"projects/{PROJECT_ID}"

    tries = 0

    while tries < 10:
        try:
            response = client.translate_text(
                    parent = parent,
                    contents= [text],
                    mime_type= "text/plain",  
                    source_language_code= "en-US",
                    target_language_code= target_language_code,
            )
            return  response.translations[0].translated_text #[translation.translated_text for translation in response.translations]
        
        except Exception as e:
                print(e)
                time.sleep(5)
                tries += 1
                continue

    return None

In [42]:
translate_table = []
for row in data:
    translate_row = []
    for cell in row:   
        if not bool(re.fullmatch(r'^[\d\W_]*$', cell)):
            translate_cell = translate_text(cell, 'zh-CN')
            translate_row.append(translate_cell)
        else:
            translate_row.append(cell)
    translate_table.append(translate_row)

In [10]:

df = pd.DataFrame(data)
header_colors = ['lightgreen', 'green', 'lightsteelblue', 'powderblue', 'sandybrown', 'lightsalmon', 'lightskyblue', 'lightgray']
background_colors = ['lightblue', 'aqua', 'cyan', 'honeydew', 'ivory', 'lemonchiffon', 'ghostwhite', 'gainsboro', 'mistyrose', 'powderblue', 'snow', 'whitesmoke', 'lime', 'lightskyblue', ]  

styled_df = (
    df.style
    .hide(axis="index") 
    .hide(axis="columns") 
    .set_table_styles([
        {'selector': 'tbody', 'props': [('background-color', random.choice(background_colors))]},  
        {'selector': 'tbody tr:nth-child(1)', 'props': [('background-color', random.choice(header_colors))]},
        {'selector': 'table', 'props': [('border', '1px solid white')]},  
        {'selector': 'td', 'props': [('min-width', '150px'), ('max-width', '450px')]}  
    ])
    .set_properties(**{
        'text-align': 'center',  # Center align text
        'font-size': '12px',  # Set font size
        'padding': '15px',  # Set padding
    })
)
import dataframe_image as dfi

# dfi.export(styled_df,f"xx.jpeg")
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib
matplotlib.style.use('ggplot')
plt.figure()

styled_df.plot()

plt.savefig('123.jpeg')  # Adjust dpi as needed

# # # Convert styled DataFrame to HTML
# html = styled_df.to_html()


# # Convert HTML to image
# imgkit.from_string(html, 'en.jpeg') 

AttributeError: 'Styler' object has no attribute 'plot'

<Figure size 640x480 with 0 Axes>

In [62]:
import os
import random
import time
from collections import defaultdict
from typing import Dict, List, Set
import requests
from sentence_splitter import split_text_into_sentences
import json
from tqdm import tqdm
import argparse
import logging
logging.basicConfig(
    level=logging.ERROR,
    format="%(asctime)s - %(levelname)s - %(message)s",
)

def inference_request(url: str, source_language: str, target_language: str, texts: List[str]) -> List[str]:

    headers = {"Content-Type": "application/json"}
    data = {
        "source_language": source_language,
        "target_language": target_language,
        "texts": texts,
    }
    response = requests.post(url, headers=headers, json=data)
    return response.json()["translated_texts"]


def call_inference_api(
    example: Dict[str, List[str]],
    url: str,
    source_language_code: str,
    target_language_code: str,
    keys_to_be_translated: List[str],
) -> Dict[str, List[str]]:

    for key in keys_to_be_translated:
        # NLLB model seems to ignore some sentences right before newline characters
        batch_str = [sen.replace('\n', '') for sen in example[key]]
        example[key] = inference_request(url, source_language_code, target_language_code, batch_str)
    return example



def translate_sent_by_sent(
    inputs
) -> Dict[str, List[str]]:

    raw_text, url, source_language_code, target_language_code = inputs
    translation = {'text': raw_text}

    keys_to_be_translated = ["text"]

    sentenized_example = defaultdict(list)

    for k in keys_to_be_translated:
        sentenized_example[f"{k}_pos"].append(0)

    for k in translation.keys():
        sentences = split_text_into_sentences(text=translation[k], language='en')
        sentenized_example[k].extend(sentences)
        sentenized_example[f"{k}_pos"].append(sentenized_example[f"{k}_pos"][-1] + len(sentences))
    
    result = call_inference_api(example=sentenized_example,
                                url=url,
                                keys_to_be_translated=keys_to_be_translated,
                                source_language_code=source_language_code,
                                target_language_code=target_language_code)
    
    for k in keys_to_be_translated:
        merged_texts = []
        l = 0
        r = 1
        while r < len(result[f"{k}_pos"]):
            start = result[f"{k}_pos"][l]
            end = result[f"{k}_pos"][r]
            merged_texts.append(' '.join(result[k][start:end]))
            l += 1
            r += 1
        translation[k] = merged_texts[0]
    
    rephrase_input = PROMPT.format(raw_text=raw_text, translation=translation['text'])

    retry_count = 0
    response_user = None
    response_chatbot = None
    while retry_count < 30:
        try:
            response_user = client.chat(
                model='command-r-plus',
                messages=[
                    {
                        "role": "user",
                        "content": rephrase_input
                    }
                ],
                temperature = 0.5,
                p = 0.9,
                max_tokens = 1024,
            )
            rephrase_output = response_user.message.content[0].text.strip()
            match_rephrase_output = re.search(r'Rephrased Translation[:：∶﹕]([\s\S]*)', rephrase_output)
            if match_rephrase_output:
                rephrase_output_extract = match_rephrase_output.group(1).strip()
            else:
                raise Exception("No match found")
            
            return rephrase_output_extract
        
        except Exception as e:
            # print(f"API Error: {e}")
            # print(f"Retry count: {retry_count}")
            # print("Retrying in 10 seconds")
            logging.error(f"API Error: {e}")
            logging.error(f"Retry count: {retry_count}")
            logging.error("Retrying in 3 seconds")
            if retry_count == 28:
                logging.error(f"Failed: {response_user}")
                logging.error(f"Failed: {response_chatbot}")
            time.sleep(3)
            retry_count += 1
    
    return None



def translate_dataset_via_inference_api(
    dataset,
    target_language_code: str,
    source_language_code: str,
    url: str = "http://localhost:8000/translate",
    output_dir: str = "./datasets",
) -> None:

    start_time = time.time()

    # translated_dataset = []
    
    # for data in tqdm(dataset):
    #     translated_dataset.append(translate_sent_by_sent((data, url, source_language_code, target_language_code)))

    translate_table = []
    for row in data:
        translate_row = []
        for cell in row:   
            if not bool(re.fullmatch(r'^[\d\W_]*$', cell)):
                # translate_cell = translate_text(cell, 'zh-CN')
                translate_cell = translate_sent_by_sent((cell, url, source_language_code, target_language_code))
                translate_row.append(translate_cell)
            else:
                translate_row.append(cell)
        translate_table.append(translate_row)
    
                           
    # print(f"Translated {len(translated_dataset)} samples")

    # with open(output_dir, "w") as f:
    #     for data in translated_dataset:
    #         f.write(json.dumps(data, ensure_ascii=False) + "\n")

    end_time = time.time()
    elapsed_time = end_time - start_time
    
    print(f"Elapsed time: {elapsed_time:.4f} seconds")
    return translate_table

# def translate_dataset(dataset_path,
#                       source_language_code: str,
#                       target_language_code: str,
#                       url: str = "http://localhost:8000/translate",
#                       output_dir: str = "./datasets",) -> None:

#     # with open(dataset_path, "r") as file:
#     #     dataset = [json.loads(line) for line in file]
#     # # print(dataset[0])

#     # print(f"Dataset size: {len(dataset)}")



#     translate_dataset_via_inference_api(
#         dataset=dataset,
#         source_language_code=source_language_code,
#         target_language_code=target_language_code,
#         url=url,
#         output_dir=output_dir,
#     )


In [63]:
table = translate_dataset_via_inference_api(dataset = None, source_language_code='eng_Latn', target_language_code = 'zho_Hans')

Elapsed time: 9.6726 seconds


In [222]:
# def to_bytes(image):
#     img_byte_arr = io.BytesIO()
#     image.convert("RGB").save(img_byte_arr, format="jpeg")
#     return img_byte_arr.getvalue()
import dataframe_image as dfi
def covert_to_table_image(table, name):

    header_colors = ['lightgreen', 'green', 'lightsteelblue', 'powderblue', 'sandybrown', 'lightsalmon', 'lightskyblue', 'lightgray', 'greenyellow', 'lightseagreen', 'lightslategray','forestgreen', 'mediumspringgreen', 'steelblue', 'mediumpurple' ]
    background_colors = ['lightblue', 'aqua', 'cyan', 'honeydew', 'ivory', 'lemonchiffon', 'ghostwhite', 'gainsboro', 'mistyrose', 'powderblue', 'snow', 'whitesmoke', 'lime', 'lightskyblue','khaki', 'mediumaquamarine', 'lightcyan', 'transparent', 'wheat']  

    df = pd.DataFrame(table)

    styled_df = (
        df.style
        .hide(axis="index")
        .hide(axis="columns")
        .set_table_styles([
            {'selector': 'tbody tr:nth-child(n+2)', 'props': [('background-color', random.choice(background_colors))]},
            {'selector': 'tbody tr:nth-child(1)', 'props': [('background-color', random.choice(header_colors))]},
            {'selector': 'table', 'props': [
                ('border', '1px solid white'),
            ]},
            {'selector': 'td', 'props': [
                ('min-width', '150px'), 
                ('max-width', '450px'),
                ('padding', '15px'),
            ]}
        ])
        .set_properties(**{
            'text-align': 'center',
            'font-size': '12px',
        })
    )

    dfi.export(styled_df,f"images/{name}.jpeg")

    with open(f'images/{name}.jpeg', "rb") as image_file:
        encoded_image = base64.b64encode(image_file.read()).decode("utf-8")

    return f"data:image/jpeg;base64,{encoded_image}"

In [230]:
# def convert_expression(expression):
#     operations = {
#         'add': '+',
#         'subtract': '-',
#         'multiply': '*',
#         'divide': '/',
#         'exp': '**',
#         'greater': '>'
#     }
    
#     if not isinstance(expression, str):
#         return str(expression)
    
#     # Recursive case: parse the expression
#     for operation, symbol in operations.items():
#         if expression.startswith(f"{operation}("):
#             # Find the content within the parentheses
#             inner_expr = expression[len(operation) + 1 : -1]  # Remove the operation and outer parentheses
#             # Split the inner expression by the first comma to separate arguments
#             arg1, arg2 = split_args(inner_expr)
#             # Recursively convert arguments
#             arg1_str = convert_expression(arg1.strip())
#             arg2_str = convert_expression(arg2.strip())
#             # Return the formatted string
#             return f"({arg1_str} {symbol} {arg2_str})"
    
#     return expression

# def split_args(inner_expr):
#     """Splits the arguments in the form 'arg1, arg2' handling nested expressions."""
#     open_parens = 0
#     for i, char in enumerate(inner_expr):
#         if char == '(':
#             open_parens += 1
#         elif char == ')':
#             open_parens -= 1
#         elif char == ',' and open_parens == 0:
#             return inner_expr[:i], inner_expr[i + 1:]
#     return inner_expr, ''


def convert_expression(program):
    operation_map = {
        'add': '+',
        'subtract': '-',
        'multiply': '*',
        'divide': '/',
        'exp': '**',
        'greater': '>',
        'table_average': 'table_average',
        'table_sum': 'table_sum',
        'table_max': 'table_max',
        'table_min': 'table_min',
    }
    operation_names = "|".join(map(re.escape, operation_map.keys()))
    pattern = rf"({operation_names})\((.*?)\)"

    split_operations = re.findall(pattern, program)
    
    # print(split_operations)
    for i, operation in enumerate(split_operations):
        op = operation[0]
        if op in ['table_average', 'table_sum', 'table_max', 'table_min']:
            split_operations[i] = f"({operation[0]}({operation[1]})"
        else:
            op_sym = operation_map[op]
            args = operation[1].split(',') #re.findall(r'\(([^()]+)\)', operation)[0].split(',')
            assert len(args) == 2
            arg1 = args[0].strip()
            arg2 = args[1].strip()
            if "#" in arg1:
                index = int(arg1.replace("#", ""))
                arg1 = split_operations[index]
            if "#" in arg2:
                index = int(arg2.replace("#", ""))
                arg2 = split_operations[index]

            split_operations[i] = f"({arg1} {op_sym} {arg2})"

    return split_operations[-1]

In [231]:
from tqdm import tqdm
with open('table_data/finqa_train.json') as f:
    finqa = json.load(f)

with open('table_data/finqa_processed.json' , 'w') as f:
    for i, data in enumerate(tqdm(finqa)):

        sample = {'Table':[], 'User': [], 'Chatbot': [], 'Image': []}
        
        sample['Table'].append(data['table'])

        User = ""
        for text in data['pre_text']:
            if text != ".":
                User += text.capitalize() + "\n"

        for text in data['post_text']:
            if text != ".":
                User += text.capitalize() + "\n"

        User += data['qa']['question'].capitalize()

        sample['User'].append({'text': User, "language": "eng_Latn", "source": "raw"})
        sample['User'].append({'text': User, "language": "eng_Latn", "source": "raw-processed"})

        Chatbot = "" 
        if data['qa']['explanation'] != "":
            Chatbot += "Rationale:" + data['qa']['explanation'].capitalize() + "\n"
        
        # computation = convert_expression(data['qa']['program_re'])
        
        computation = convert_expression(data['qa']['program'])

        if "const" in computation:
            computation = computation.replace("const_", "")
            if "m1" in computation:
                computation = computation.replace("m1", "-1")
            
        Chatbot += "Computations: " +  computation  + "\n"
        
        answer = data['qa']['exe_ans']
        try:
            if data['qa']['answer'] != "":
                if ('%' in data['qa']['answer']) and (type(answer) == float):
                    if round(float(data['qa']['answer'].replace('%', ''))/100,2) == round(answer, 2):
                        answer = data['qa']['answer']
        except:
            print(f"Error in {i}")
            print(data['qa']['answer'])
            print(answer)
            pass

        Chatbot += "Answer: " + str(answer)

        sample['Chatbot'].append({'text': Chatbot, "language": "eng_Latn", "source": "raw"})

        sample['Image'].append(covert_to_table_image(data['table'], i))

        f.write(json.dumps(sample, ensure_ascii=False) + "\n") 

  4%|▍         | 279/6251 [06:14<2:07:37,  1.28s/it]

Error in 279
yes , 38.2%
0.38166


 10%|▉         | 617/6251 [14:09<2:13:07,  1.42s/it]

Error in 617
8.5%\\n
0.085


 21%|██        | 1309/6251 [30:34<2:03:57,  1.50s/it]

...

100%|█████████▉| 6239/6251 [2:34:17<00:16,  1.39s/it]

Error in 6239
17.1% increase
2556557.0


100%|█████████▉| 6245/6251 [2:34:25<00:08,  1.37s/it]

Error in 6245
9.6%\\n
0.09615


100%|██████████| 6251/6251 [2:34:35<00:00,  1.48s/it]


In [225]:
type(0.20886)

float

In [233]:
import json

with open('table_data/MultiHiertt_processed.jsonl', 'r') as f, open('table_data/MultiHiertt_processed_2.jsonl', 'w') as output_file:
    for i, line in enumerate(f):
        data = json.loads(line)
        # data['command_id'] = f"MultiHiertt-{i}"
        # data['metadata'] = {"source": "MultiHiertt"}
        # data['index'] = i
        
        output_file.write(json.dumps(data, ensure_ascii=False) + "\n")


In [252]:
import json
import pandas as pd
with open('table_data/MultiHiertt_processed.jsonl', 'r') as f, open('table_data/MultiHiertt_processed_2.jsonl', 'w') as output_file:
    for i, line in enumerate(f):
        data = json.loads(line)
        table_list= []
        tables = data['Table'][0]
        for table in tables:
            table_list.append(pd.read_html(table)[0].fillna("").values.tolist())
        data['Table'] = table_list
        output_file.write(json.dumps(data, ensure_ascii=False) + "\n")


In [1]:
import json
with open('/home/olivernan_cohere_com/gpt_recaption_data_2024_11_01_raw/RecapMultiHiertt/train.jsonl', 'r') as f:
    for i, line in enumerate(f):
        data = json.loads(line)

In [253]:
with open('table_data/MultiHiertt_processed.jsonl', 'r') as f:
    data = [json.loads(line) for line in f]

In [256]:
(data[0]['Table'])

[[['',
   'December 31, 2007',
   'December 31, 2007',
   'December 31, 2006',
   'December 31, 2006'],
  ['', 'Carrying Value', 'Fair Value', 'Carrying Value', 'Fair Value'],
  ['', '(In thousands)', '(In thousands)', '(In thousands)', '(In thousands)'],
  ['Mortgages and notes payable',
   '$584,795',
   '$603,200',
   '$420,061',
   '$449,130'],
  ['Senior notes', '$977,556', '$979,562', '$1,127,508', '$1,146,767']],
 [['', 2014, 2013],
  ['Shares outstanding at beginning of period', 1945, 1974],
  ['Treasury stock purchases-1', -46, -27],
  ['Other-2', 52, -2],
  ['Shares outstanding at end of period', 1951, 1945]],
 [['',
   'Year Ended December 31, 2010',
   'Year Ended December 31, 2010',
   'Year Ended December 31, 2010',
   'Year Ended December 31, 2010',
   'Year Ended December 31, 2010'],
  ['',
   'Other Assets',
   'Separate Account Assets -4',
   'Future Policy Benefits',
   'Long- term Debt',
   'Other Liabilities'],
  ['',
   '(in millions)',
   '(in millions)',
   '(in

In [249]:
import pandas as pd

df = pd.read_html(data[0]['Table'][0][0])[0].fillna("")
df.values.tolist()

[['',
  'December 31, 2007',
  'December 31, 2007',
  'December 31, 2006',
  'December 31, 2006'],
 ['', 'Carrying Value', 'Fair Value', 'Carrying Value', 'Fair Value'],
 ['', '(In thousands)', '(In thousands)', '(In thousands)', '(In thousands)'],
 ['Mortgages and notes payable',
  '$584,795',
  '$603,200',
  '$420,061',
  '$449,130'],
 ['Senior notes', '$977,556', '$979,562', '$1,127,508', '$1,146,767']]

In [250]:
df

,0,1,2,3,4
0,,"December 31, 2007","December 31, 2007","December 31, 2006","December 31, 2006"
1,,Carrying Value,Fair Value,Carrying Value,Fair Value
2,,(In thousands),(In thousands),(In thousands),(In thousands)
3,Mortgages and notes payable,"$584,795","$603,200","$420,061","$449,130"
4,Senior notes,"$977,556","$979,562","$1,127,508","$1,146,767"


In [7]:
import os
for lang in os.listdir('/home/olivernan_cohere_com/recap_data_translation_2024_11_01_raw/RecapMultiHiertt_translation'):
    with open(f'/home/olivernan_cohere_com/recap_data_translation_2024_11_01_raw/RecapMultiHiertt_translation/{lang}/train.jsonl', 'r') as f:
        line_number = [_ for _ in f]
        print(f"{lang}: {len(line_number)}")

ukr_Cyrl: 7830
arb_Arab: 7830
ell_Grek: 7830
por_Latn: 7830
ind_Latn: 7830
deu_Latn: 7830
ces_Latn: 7830
hin_Deva: 7830
nld_Latn: 7830
pes_Arab: 7830
vie_Latn: 7830
pol_Latn: 7830
rus_Cyrl: 7830
kor_Hang: 7830
zho_Hant: 7830
fra_Latn: 7830
spa_Latn: 7830
tur_Latn: 7830
jpn_Jpan: 7830
zho_Hans: 7830
heb_Hebr: 7830
ron_Latn: 7830
ita_Latn: 7830
